In [2]:
from newspaper import Article #newspaper has some nlp stuff, don't know if it is good enough

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [3]:
#https://towardsdatascience.com/scrape-and-summarize-news-articles-in-5-lines-of-python-code-175f0e5c7dfc
url = "https://thenewstack.io/dont-mess-with-the-master-working-with-branches-in-git-and-github/"
article = Article(url, language='en', fetch_images=False)

#download and parse the article
article.download()
article.parse()
#article.nlp() #Don't know if we need to parse

text = article.text

In [4]:
doc = nlp(text)
#displacy.render(nlp(str(sentences[20])), jupyter=True, style='ent')

labels = [x.label_ for x in doc.ents]
Counter(labels)

Counter({'ORG': 14,
         'GPE': 2,
         'CARDINAL': 11,
         'NORP': 4,
         'ORDINAL': 2,
         'WORK_OF_ART': 3,
         'PERSON': 1})

In [5]:
sentences = [x for x in doc.sents]
print(sentences[21])

Instead, everyone uses branches created from master to experiment, make edits and additions and changes, before eventually rolling that branch back into the master once they have been approved and are known to work.


In [6]:
items = [x.text for x in doc.ents]
Counter(items).most_common(3)

[('hello_octo', 10), ('GitHub', 9), ('one', 3)]

In [18]:
! python3 -m nltk.downloader wordnet
! python3 -m nltk.downloader omw
! pip3 install spacy-wordnet
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
from spacy_wordnet.wordnet_annotator import WordnetAnnotator

/Users/Sam/anaconda3/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /Users/Sam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/Sam/anaconda3/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package omw to /Users/Sam/nltk_data...
[nltk_data]   Package omw is already up-to-date!


ValueError: [E007] 'WordnetAnnotator' already exists in pipeline. Existing names: ['tagger', 'WordnetAnnotator', 'parser', 'ner']

In [17]:

for (item, count) in Counter(items).most_common():
    token = nlp(item)[0]
    print(token)
    print(token._.wordnet.synsets())
    #print(token._.wordnet.lemmas())
    #print(token._.wordnet.wordnet_domains())
    print()


hello_octo
[]

GitHub
[]

one
[Synset('one.n.01'), Synset('one.n.02'), Synset('one.s.01'), Synset('one.s.02'), Synset('one.s.03'), Synset('one.s.04'), Synset('one.s.05'), Synset('one.s.06'), Synset('matchless.s.01')]

Github.com
[]

more
[Synset('more.a.01'), Synset('more.a.02')]

One
[Synset('one.n.01'), Synset('one.n.02'), Synset('one.s.01'), Synset('one.s.02'), Synset('one.s.03'), Synset('one.s.04'), Synset('one.s.05'), Synset('one.s.06'), Synset('matchless.s.01')]

1
[Synset('one.n.01'), Synset('one.s.01')]

first
[Synset('first.r.01'), Synset('first.r.02'), Synset('first.r.03'), Synset('foremost.r.01')]

second
[Synset('second.s.01'), Synset('second.a.02')]

2
[Synset('two.n.01'), Synset('two.s.01')]

Hello
[Synset('hello.n.01')]

two
[Synset('two.n.01'), Synset('deuce.n.04'), Synset('two.s.01')]

git
[Synset('rotter.n.01')]

git
[Synset('rotter.n.01')]

Git
[Synset('rotter.n.01')]

3
[Synset('three.n.01'), Synset('three.s.01')]

4
[Synset('four.n.01'), Synset('four.s.01')]

5
[Sy

In [ ]:
#https://stackoverflow.com/questions/22031968/how-to-find-distance-between-two-synset-using-python-nltk-in-wordnet-hierarchy
# want to do this next